## Siamese BERT-networks for semantic searching / information retrieval

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
from datasets import load_dataset

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

from random import sample, seed, shuffle
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader

In [2]:
PERSON = 'Sinan Ozdemir'

# Note this is NOT an efficient way to search on google. This is done simply for education purposes
google_html = BeautifulSoup(requests.get(f'https://www.google.com/search?q={PERSON}').text).get_text()[:1024]

nlp = pipeline('question-answering', 
               model='deepset/roberta-base-squad2', 
               tokenizer='deepset/roberta-base-squad2', 
               max_length=10)

nlp(f'Who is {PERSON}?', google_html)

{'score': 0.09814480692148209,
 'start': 545,
 'end': 591,
 'answer': 'data scientist, start-up founder, and educator'}

In [3]:
PERSON = 'Barack Obama'

# Note this is NOT an efficient way to search on google. This is done simply for education purposes
google_html = BeautifulSoup(requests.get(f'https://www.google.com/search?q={PERSON}').text).get_text()[:1024]

nlp = pipeline('question-answering', 
               model='deepset/roberta-base-squad2', 
               tokenizer='deepset/roberta-base-squad2', 
               max_length=10)

nlp(f'Who is {PERSON}?', google_html)

{'score': 0.06925773620605469,
 'start': 519,
 'end': 573,
 'answer': 'ObamaBarack Hussein Obama II is an American politician'}

In [4]:
# textbook about insects
text = urlopen('https://www.gutenberg.org/cache/epub/10834/pg10834.txt').read().decode()

# Only keep documents of at least 100 characters
documents = list(filter(lambda x: len(x) > 100, text.split('\r\n\r\n')))

documents = np.array(documents)

print(f'There are {len(documents)} documents/paragraphs')

There are 79 documents/paragraphs


In [5]:
# This model pre-trained on an asymmetric semantic search task
# We use the Bi-Encoder to encode all the documents, so that we can use it with sematic search
bi_encoder = SentenceTransformer('msmarco-distilbert-base-v4')
bi_encoder.max_seq_length = 256     # Truncate long documents to 256 tokens

bi_encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [6]:
# Documents are encoded by calling model.encode(). This takes about 25 seconds on my laptop
document_embeddings = bi_encoder.encode(documents, convert_to_tensor=True, show_progress_bar=True)

document_embeddings.shape

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([79, 768])

In [7]:
QUESTION = 'How many horns does a flea have?'  # a natural language query

In [8]:
# Encode the query using the bi-encoder and find relevant documents
question_embedding = bi_encoder.encode(QUESTION, convert_to_tensor=True)

print(question_embedding.shape)

# Number of documents to retrieve with the bi-encoder
hits = util.semantic_search(question_embedding, document_embeddings, top_k=3)[0]

hits

torch.Size([768])


[{'corpus_id': 14, 'score': 0.489949107170105},
 {'corpus_id': 19, 'score': 0.24793732166290283},
 {'corpus_id': 21, 'score': 0.18478815257549286}]

In [9]:
print(f'Question: {QUESTION}\n')

for i, hit in enumerate(hits):
    
    print(f'Document {i + 1} Cos_Sim {hit["score"]:.3f}:\n\n{documents[hit["corpus_id"]]}')
    print('\n')

Question: How many horns does a flea have?

Document 1 Cos_Sim 0.490:

When examined by a microscope, the flea is a pleasant object. The body
is curiously adorned with a suit of polished armour, neatly jointed, and
beset with a great number of sharp pins almost like the quills of a
porcupine: it has a small head, large eyes, two horns, or feelers, which
proceed from the head, and four long legs from the breast; they are very
hairy and long, and have several joints, which fold as it were one
within another.


Document 2 Cos_Sim 0.248:

The Chego is a very small animal, about one fourth the size of a common
flea: it is very troublesome, in warm climates, to the poor blacks, such
as go barefoot, and the slovenly: it penetrates the skin, under which it
lays a bunch of eggs, which swell to the bigness of a small pea.


Document 3 Cos_Sim 0.185:


This is one of the largest of the insect tribe. It is met with in
different countries, and of various sizes, from two or three inches to
nearly a 

In [10]:
# answer the question from the top document
nlp(QUESTION, str(documents[hits[0]['corpus_id']]))

{'score': 0.8524739146232605, 'start': 259, 'end': 262, 'answer': 'two'}

In [11]:
# This is called an "Open Book Q/A" System

In [12]:
# load up the adversarial_qa dataset from the Q/A use-case
training_qa = load_dataset('adversarial_qa', 'adversarialQA', split='train')

good_training_data = []
bad_training_data = []
    
last_example = None
for example in training_qa:
    if last_example and example['context'] != last_example['context']:
        bad_training_data.append((example['question'], last_example['context'], 0.0))  #  add neutral examples
    # question, context, label is 1 if should be matched together
    good_training_data.append((example['question'], example['context'], 1.0))
    last_example = example

Reusing dataset adversarial_qa (/Users/sinanozdemir/.cache/huggingface/datasets/adversarial_qa/adversarialQA/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


In [13]:
len(good_training_data), len(bad_training_data)

(30000, 2647)

In [14]:
good_training_data[-1]

('What letter designates what Ektachrome is designed for?',
 'Some high-speed black-and-white films, such as Ilford Delta 3200 and Kodak T-MAX P3200, are marketed with film speeds in excess of their true ISO speed as determined using the ISO testing method. For example, the Ilford product is actually an ISO 1000 film, according to its data sheet. The manufacturers do not indicate that the 3200 number is an ISO rating on their packaging. Kodak and Fuji also marketed E6 films designed for pushing (hence the "P" prefix), such as Ektachrome P800/1600 and Fujichrome P1600, both with a base speed of ISO 400.',
 1.0)

In [15]:
bad_training_data[-1]

('What film beside Ektachrome and Fujichorme is designed for pushing?',
 'The Weston Cadet (model 852 introduced in 1949), Direct Reading (model 853 introduced 1954) and Master III (models 737 and S141.3 introduced in 1956) were the first in their line of exposure meters to switch and utilize the meanwhile established ASA scale instead. Other models used the original Weston scale up until ca. 1955. The company continued to publish Weston film ratings after 1955, but while their recommended values often differed slightly from the ASA film speeds found on film boxes, these newer Weston values were based on the ASA system and had to be converted for use with older Weston meters by subtracting 1/3 exposure stop as per Weston\'s recommendation. Vice versa, "old" Weston film speed ratings could be converted into "new" Westons and the ASA scale by adding the same amount, that is, a film rating of 100 Weston (up to 1955) corresponded with 125 ASA (as per ASA PH2.5-1954 and before). This conver

In [16]:
# https://www.sbert.net/docs/training/overview.html for more information on training

seed(42)  # seed our upcoming sample

sampled_training_data = sample(good_training_data, 1000) + sample(bad_training_data, 1000)

shuffle(sampled_training_data)  # shuffle our data around

training_index = int(.8 * len(sampled_training_data))  # Get an 80/20 train/test split

In [17]:
# Define the training examples
train_examples = [InputExample(texts=t[:2], label=t[2]) for t in sampled_training_data[:training_index]]

train_examples[0].__dict__

{'guid': '',
 'texts': ("Why might I question Tito's ethics?",
  'In 2013 a lot of media coverage was given to unclassified NSA\'s study in Cryptologic Spectrum that concluded that Tito did not speak the language as a native, and had features of other Slavic languages (Russian and Polish). The hypothesis that "a non-Yugoslav, perhaps a Russian or a Pole" assumed Tito\'s identity was included. The report also notes Draža Mihailović\'s impressions of Tito\'s Russian origins.'),
 'label': 0.0}

In [18]:
# Define the train dataset, a dataloader and the train loss
# A data loader is the object that specifically shuffles/grabs batches of data from a Dataset
# We don't usually have to explicitly create one using the Trainer because it has a default loader built in
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

train_loss = losses.CosineSimilarityLoss(bi_encoder)

In [19]:
# Evaluation data, sentences1 and sentences2 are lists of questions and context respectively and scores are 0 or 1
sentences1, sentences2, scores = zip(*sampled_training_data[training_index:])

# evaluator will evaluate embedding closeness
evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

In [20]:
bi_encoder.evaluate(evaluator)  # initial evalaution (higher embedding similarity is better)

0.4705443607475632

In [ ]:
# Fine-tune the model using the fit method
bi_encoder.fit(
    train_objectives=[(train_dataloader, train_loss)], 
    output_path='ir/results',
    epochs=2, 
    evaluator=evaluator
)

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
bi_encoder.evaluate(evaluator)  # final evalaution (higher embedding similarity is better)
# Not a huge jump in performance with 2 epochs. We could try more data or more epochs

In [ ]:
# load fine-tuned IR model
finetuned_bi_encoder = SentenceTransformer('ir/results')

In [ ]:
# Slightly more confident results!

document_embeddings = finetuned_bi_encoder.encode(documents, convert_to_tensor=True, show_progress_bar=True)

question_embedding = finetuned_bi_encoder.encode(QUESTION, convert_to_tensor=True)

# Get document hits
hits = util.semantic_search(question_embedding, document_embeddings, top_k=3)[0]

print(f'Question: {QUESTION}\n')

for i, hit in enumerate(hits):
    
    print(f'Document {i + 1} Cos_Sim {hit["score"]:.3f}:\n\n{documents[hit["corpus_id"]]}')
    print('\n')

In [ ]:
def gutenberg_to_documents(gutenberg_url, bi_encoder):
    text = urlopen(gutenberg_url).read().decode()
    documents = np.array(list(filter(lambda x: len(x) > 100, text.split('\r\n\r\n'))))
    print(f'There are {len(documents)} documents/paragraphs')
    return documents, bi_encoder.encode(documents)

def retrieve_relevant_documents(bi_encoder, query, documents, document_embeddings, hits=3):
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)

    hits = util.semantic_search(query_embedding, document_embeddings, top_k=hits)[0]

    for i, hit in enumerate(hits):
        print(f'Document {i + 1} Cos_Sim {hit["score"]:.3f}:\n\n{documents[hit["corpus_id"]]}')
        print('\n')
    print(f"Answer from Top Document: {nlp(query, str(documents[hits[0]['corpus_id']]))}")

In [ ]:
banks_to_bassoon_documents, banks_to_bassoon_embeddings = gutenberg_to_documents(
    'https://www.gutenberg.org/cache/epub/27480/pg27480.txt', finetuned_bi_encoder
)

In [ ]:
retrieve_relevant_documents(finetuned_bi_encoder,
    'What is a banshee?', banks_to_bassoon_documents, banks_to_bassoon_embeddings, 2
)

In [ ]:
retrieve_relevant_documents(finetuned_bi_encoder,
    'When was the Imperial Bank of Germany founded?', banks_to_bassoon_documents, banks_to_bassoon_embeddings, 2
)